In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, auc

from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar

import numpy as np
import pandas as pd

### **Step 01: Data Wrangling

Imported data sets here are about the third repurchase rate (from second to third purchase) of several cohorts of the customers in four different types of Order Dynamics and three different brands. The data set has not been updated since November 2021.

Cohorts are based on the first purchase month of each customer. All the customers who made the first purchase in certain brand in the same month are grouped in one cohort.

Order Dynamics: composed of Onetime Order and Subsription
- OO: Onetime Order -> Onetime Order
- OS: Onetime Order -> Subscription
- SS: Subscription -> Subscription
- SO: Subscirption -> Subscription

In [3]:
# set base dir
# import .csv file 
# loaded csv file is about repurchase rate in the Subscription to Subscription Order Dynamics of all cohorts
# by loading different files, we can conduct the prediction of the other types of Order Dynamics
df_RR = pd.read_csv('YOUR_PURCHASING_DATA.csv', index_col=0)

In [4]:
df_RR.head()

,avgDaysPast,N2pop,N3pop,hist_RR,d0,d1,d2,d3,d4,d5,...,d711,d712,d713,d714,d715,d716,d717,d718,d719,d720
cohort,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1180.53,234,8,3.42%,0.43%,0.43%,0.43%,0.43%,0.43%,0.43%,...,3.15%,3.15%,3.15%,3.15%,3.15%,3.15%,3.15%,3.15%,3.15%,3.15%
2018-02-01,1084.49,3246,224,6.90%,0.62%,0.68%,0.74%,0.80%,0.92%,1.02%,...,6.74%,6.74%,6.74%,6.78%,6.78%,6.78%,6.78%,6.78%,6.78%,6.79%
2018-03-01,1080.73,5134,350,6.82%,0.80%,0.93%,1.07%,1.09%,1.38%,1.54%,...,6.76%,6.76%,6.76%,6.76%,6.76%,6.74%,6.74%,6.74%,6.74%,6.75%
2018-04-01,1059.82,6034,398,6.60%,0.76%,0.85%,0.91%,1.03%,1.16%,1.29%,...,6.67%,6.68%,6.68%,6.66%,6.67%,6.67%,6.67%,6.67%,6.68%,6.65%
2018-05-01,1040.01,6507,427,6.56%,0.68%,0.72%,0.78%,0.88%,0.92%,0.98%,...,6.39%,6.40%,6.40%,6.40%,6.40%,6.40%,6.40%,6.40%,6.40%,6.41%


In [6]:
# preprocess DataFrame for use
avgDaysPast = df_RR.avgDaysPast.to_dict()
df_RR = df_RR.loc[:, 'd0':'d420'].transpose()
df_RR.replace('[%]', r'', regex=True, inplace=True)
df_RR = df_RR.apply(pd.to_numeric)
df_RR.index.name = 'days cnt'

# remove cohorts with abnormal pattern in Repurchase Rate change
# df_RR.drop(columns='2021-06-01', axis=1, inplace=True)
df_RR.drop(columns='2021-07-01', axis=1, inplace=True)
df_RR.drop(columns='2021-08-01', axis=1, inplace=True)
df_RR.drop(columns='2021-09-01', axis=1, inplace=True)
df_RR.drop(columns='2021-10-01', axis=1, inplace=True)
df_RR.reset_index(drop=True, inplace=True)

In [7]:
#datetime.strptime("2021-01-01", '%Y-%m-%d')+relativedelta(month = 2)-relativedelta(days=1)
df_RR.head()

cohort,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,...,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01
0,0.43,0.62,0.80,0.76,0.68,0.72,0.58,0.57,0.78,0.82,...,0.21,0.17,0.11,0.16,0.20,0.15,0.14,0.31,0.4,0.1
1,0.43,0.68,0.93,0.85,0.72,0.81,0.62,0.62,0.87,0.95,...,0.23,0.21,0.15,0.16,0.26,0.22,0.21,0.50,0.4,0.1
2,0.43,0.74,1.07,0.91,0.78,0.92,0.68,0.65,0.91,1.05,...,0.32,0.30,0.20,0.21,0.33,0.22,0.28,0.50,0.4,0.2
3,0.43,0.80,1.09,1.03,0.88,1.01,0.72,0.73,1.01,1.13,...,0.34,0.34,0.20,0.26,0.39,0.30,0.35,0.50,0.8,0.2
4,0.43,0.92,1.38,1.16,0.92,1.11,0.76,0.74,1.05,1.21,...,0.36,0.34,0.23,0.32,0.39,0.45,0.35,0.62,0.9,0.2


In [8]:
# replace `NA` null-values with the current date
cohort_all = list(df_RR.columns)
date_cohort = []
now = datetime.now()

# Calculate `days_past` value 
for i, cohort in enumerate(cohort_all):
    #print(cohort)
    #define "avgdp30mult" value as the smallest multiples of 30 closest to "avgDaysPast" value
    # (since it is the period with the most stable Repurchase Rate)
    avgdp30mult = int((avgDaysPast[cohort]//30)*30)

    # set `NA` null-values for the df_RR DataFrame
    if avgdp30mult < 420:
        df_RR.iloc[avgdp30mult:, i] = np.nan

df_RR_prefill = df_RR.copy(deep=True)

### **Step 02: `Curve-Fitting` for Base Cohorts**

In [9]:
# Fitting Function - rational function
def func_rational(x, a, b, c, d):
    return np.polyval([a,b], x) / np.polyval([c,d], x)

# take arbitrary values as a initial settings of curve-fitting parameters
initialGuess_rational = [0.1, 0.1, 0.1, 0.1]

In [10]:
# store the fitted base cohorts into a new data frame "df_RR_basefit"
df_RR_basefit = pd.DataFrame()

In [11]:
# Custom curve-fitting function for all base cohorts with over 720 days past 
# (regard "720 days = about 2 years" as desertion period
def fit_basecohort():
    cohort_base = list(df_RR.loc[:, (df_RR.isnull().sum() == 0)].columns)
    list_days = list(range(0, 421))

    # empty df_RR_basefit 
    df_RR_basefit.drop(index=df_RR_basefit.index, inplace=True)
    
    # Set df_RR_basefit
    df_RR_basefit['cohort'] = cohort_base
    df_RR_basefit.set_index('cohort', inplace=True)

    for cohort in cohort_base:
        # set up X,Y Data
        xBase = list_days
        yBase = df_RR.loc[:, cohort]
        
        # Perform Curve-fitting 
        popt_base, pcov_base = curve_fit(func_rational, xBase, yBase, initialGuess_rational)
        # Based on this point, we can adjust the curve-fitting parameters

        # Update DataFrame with fitted parameters
        df_RR_basefit.loc[cohort, 'fitted_a'] = popt_base[0]
        df_RR_basefit.loc[cohort, 'fitted_b'] = popt_base[1]
        df_RR_basefit.loc[cohort, 'fitted_c'] = popt_base[2]
        df_RR_basefit.loc[cohort, 'fitted_d'] = popt_base[3]

### **Step 03: `Fill` the null-values in each cohort with curve-fitting method**

In [12]:
# Select all cohorts that needs to be filled 
cohort_fill = list(df_RR.loc[:, (df_RR.isnull().sum() > 0)].columns)

# Create DataFrame with trimmed Repurchase Rate data 
df_RR_shortTerm = pd.DataFrame()

# Create DataFrame for Short-term Repurchase Rate Comparison
df_RR_compare = pd.DataFrame({'fit_cohort':cohort_fill, 'UB_cohort':'', 'UB_MSE':10000, 'LB_cohort':'', 'LB_MSE':10000, 'UB_SUB_cohort':'', 'UB_SUB_MSE':10000, 'LB_SUB_cohort':'', 'LB_SUB_MSE':10000})
df_RR_compare.set_index('fit_cohort', inplace=True)

# Optimize the variables with weight tuning 
tuning_range = 0.01 
tuning_external_range = 4
optimal_param = [0.0, 0.0, 0.0, 0.0]

In [13]:
# Loop & Compare by MSE and AUC (Mean-Squared-Error and Area-Under-Curve)
def compare_shortTerm(fill_cohort, compare_cohort, fit_days, AUC_my):

    dFit = np.linspace(0, fit_days, fit_days+1)

    # Update df_RR_compare based on AUC and lowest MSE
    for compare in compare_cohort:
        # Calculate AUC and MSE of the comparing cohorts       
        AUC_compare = auc(dFit, df_RR.loc[:days_tofit, compare]) 
        MSE = mean_squared_error(df_RR.loc[:fit_days, fill_cohort], df_RR_shortTerm[compare])
        #print("Cohort :", compare, " AUC :", AUC_compare, " MSE :",MSE)

        # Check the cohorts near the upper-bound
        if AUC_compare > AUC_my:
            if MSE < df_RR_compare.loc[fill_cohort, 'UB_MSE']:
                # Push current UB to UB_SUB
                df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE'] = df_RR_compare.loc[fill_cohort, 'UB_MSE']
                df_RR_compare.loc[fill_cohort, 'UB_SUB_cohort'] = df_RR_compare.loc[fill_cohort, 'UB_cohort']
                # Update UB
                df_RR_compare.loc[fill_cohort, 'UB_MSE'] = MSE
                df_RR_compare.loc[fill_cohort, 'UB_cohort'] = compare
            else:
                # For the case when the new MSE is larger than UB_MSE -> Push new MSE in SUB_MSE if new MSE is less than SUB_MSE
                if MSE < df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE']:
                    df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE'] = MSE
                    df_RR_compare.loc[fill_cohort, 'UB_SUB_cohort'] = compare
                else : 
                    pass
        # Check the cohorts near lower-bound
        elif AUC_compare < AUC_my:
            if MSE < df_RR_compare.loc[fill_cohort, 'LB_MSE']:
                # Push current LB to LB_SUB
                df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE'] = df_RR_compare.loc[fill_cohort, 'LB_MSE']
                df_RR_compare.loc[fill_cohort, 'LB_SUB_cohort'] = df_RR_compare.loc[fill_cohort, 'LB_cohort']
                # Update LB
                df_RR_compare.loc[fill_cohort, 'LB_MSE'] = MSE
                df_RR_compare.loc[fill_cohort, 'LB_cohort'] = compare
            else:
                # For the case when the new MSE is greater than LB_MSE -> Push new MSE in SUB_MSE if new MSE is less than SUB_MSE
                if MSE < df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE']:
                    df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE'] = MSE
                    df_RR_compare.loc[fill_cohort, 'LB_SUB_cohort'] = compare
                else : 
                    pass
        

In [19]:
# Define customized MSE calculation function based on the given weighted values
def get_weighted_MSE(base_cohort, fit_day, weight_1, weight_2):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']

    # Highest Cohort 
    if cohort_UB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'LB_SUB_cohort']
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_LB]) + (weight_2 * df_RR_shortTerm[cohort_SUB])
    # Lowest Cohort
    elif cohort_LB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'UB_SUB_cohort']
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_UB]) + (weight_2 * df_RR_shortTerm[cohort_SUB])
    # Normal
    else:
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_UB]) + (weight_2 * df_RR_shortTerm[cohort_LB])
    
    return mean_squared_error(df_RR.loc[:fit_day, base_cohort], weighted_y)

# Define customized weighted parameter function based on the given weights
def get_weighted_param(base_cohort, weight_1, weight_2):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']
    
    # Highest Cohort 
    if cohort_UB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'LB_SUB_cohort']
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_LB, :]) + (weight_2 * df_RR_basefit.loc[cohort_SUB, :])
    # Lowest Cohort
    elif cohort_LB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'UB_SUB_cohort']
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_SUB, :]) + (weight_2 * df_RR_basefit.loc[cohort_UB, :])
    # Normal
    else:
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_UB, :]) + (weight_2 * df_RR_basefit.loc[cohort_LB, :])
    
    return weighted_param

In [14]:
# Loop & find the optimal parameter for Repurchase Rate prediction
def optimize_parameter(base_cohort, fit_day):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']
    optimal_MSE, optimal_weight = 10000, [0.0, 0.0]

    # Case: Normal 
    if (cohort_UB != '') & (cohort_LB != ''):
        for i in range(int(1/tuning_range+1)):
            test_weight = [(i*tuning_range), (1-(i*tuning_range))]
            test_MSE = get_weighted_MSE(base_cohort, fit_day, *test_weight)
            
            # Update the optimal values
            if test_MSE < optimal_MSE:
                optimal_MSE = test_MSE
                optimal_weight = test_weight[:]    
    
    # Case: Highest or Lowest 
    else:
        for i in range(int(1/tuning_range+1)*(tuning_external_range-1)):
            # Case: Highest
            if cohort_UB == '':
                test_weight = [(tuning_external_range-i*tuning_range), (1-(tuning_external_range-i*tuning_range))]
            # Case: Lowest
            elif cohort_LB == '':
                test_weight = [(1-(tuning_external_range-i*tuning_range)), (tuning_external_range-i*tuning_range)]

            test_MSE = get_weighted_MSE(base_cohort, fit_day, *test_weight)
            
            # Update the optimal values
            if test_MSE < optimal_MSE:
                optimal_MSE = test_MSE
                optimal_weight = test_weight[:] 

    #print('optimal_weight:\t', optimal_weight, '\t', 'optimal_MSE:\t', optimal_MSE)
    return get_weighted_param(base_cohort, *optimal_weight)

In [15]:
# update 'null' value in cohorts with fitted parameters 
def update_nullValue(fill_cohort, fit_day, optimalparam):
    
    # Pass optimal paratmeter to rational_function
    xFit = np.linspace(0, 420, 421)
    yPred = func_rational(xFit, *optimalparam)

    # update `null` value in cohorts 
    df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):]

    chk = (df_RR.loc[fit_day, fill_cohort] - yPred[(fit_day+1)])

    if (chk >= 0) & (abs(chk) > 0.1):
        df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):] + abs(chk)
    
    elif (chk < 0) & (abs(chk) > 0.1):
        df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):] - abs(chk)

In [20]:
# Loop & Fill all the null values in the cohorts
for fill in cohort_fill:
    # reset the base cohort before every update process
    fit_basecohort()

    # Set up the cohort & days range for the computation
    cohort_compare = df_RR_basefit.index.values
    days_tofit = (420 - df_RR[fill].isnull().sum())
    dFit = np.linspace(0, days_tofit, days_tofit+1)

    # Update df_RR_shortTerm 
    df_RR_shortTerm = df_RR.loc[:days_tofit, cohort_compare]
    AUC_my = auc(dFit, df_RR.loc[:days_tofit, fill]) 
    print(AUC_my)
    compare_shortTerm(fill, cohort_compare, days_tofit, AUC_my)
    
    # Optimize the weighted values on the parameters
    optimal_param = optimize_parameter(fill, days_tofit)

    # update 'null' value in cohorts with fitted parameters 
    update_nullValue(fill, days_tofit, optimal_param)

1583.8049999999998
1292.52
1149.9499999999998
793.55
807.02
501.38
485.0950000000001
469.35
412.405
437.305
360.51
296.41999999999996
193.95999999999998
154.09999999999997
41.29


In [21]:
df_RR_compare.head()

,UB_cohort,UB_MSE,LB_cohort,LB_MSE,UB_SUB_cohort,UB_SUB_MSE,LB_SUB_cohort,LB_SUB_MSE
fit_cohort,,,,,,,,
2020-04-01,2018-07-01,0.077302,2018-08-01,0.124630,2018-07-01,0.077302,2018-08-01,0.124630
2020-05-01,2018-08-01,0.057362,2018-01-01,2.825148,2020-04-01,0.212466,,10000.000000
2020-06-01,2020-05-01,0.033616,2018-01-01,2.742510,2018-08-01,0.072190,,10000.000000
2020-07-01,2020-06-01,0.590305,2018-01-01,0.694930,2020-05-01,0.703385,,10000.000000
2020-08-01,2020-06-01,0.483520,2020-07-01,0.038673,2020-05-01,0.579978,2018-01-01,0.816728


In [22]:
fig_raw = px.line(data_frame=df_RR_prefill)
fig_raw.show()
#fig_raw.write_html("SubscriptionToOnetime_RepurchaseSecondToThird_Raw.html")

In [23]:
fig_pred = px.line(data_frame=df_RR)
fig_pred.show()
#fig_pred.write_html("SubscriptionToSubcription_RepurchaseSecondToThird_Predicted.html")

In [133]:
#df_RR.iloc[420].to_csv("SubscriptionToOnetime_RepurchaseSecondToThird_Predicted.csv", encoding= "cp949")

In [24]:
px.line(data_frame=df_RR)

In [27]:
# Save predicted results as .csv format
today = datetime.today()
today = today.strftime('%Y%m%d')
df_RR_save = df_RR.loc[510: :].transpose()
#df_RR_save.to_csv('LTV_Pred_Result'+today+'.csv')

# df_RR.loc[720, :].to_csv('LTV_Pred_Result'+today+'.csv')